In [3]:
'''dataloader检查 必须num_workers=0'''
for images, labels in dataloader:
    print(f"图像形状: {images.shape}")
    print(f"标签: {labels}")
    break

# 获取原始标签示例
if len(dataset) > 0:
    original_label = dataset.get_original_label(0)
    print(f"第一个样本的原始标签: {original_label}")

图像形状: torch.Size([1000, 1, 28, 28])
标签: tensor([1, 3, 7, 0, 3, 6, 3, 9, 2, 7, 4, 8, 1, 2, 3, 9, 3, 6, 8, 1, 0, 0, 0, 7,
        6, 7, 8, 3, 0, 7, 2, 3, 8, 2, 5, 1, 5, 8, 8, 5, 8, 0, 1, 9, 0, 8, 6, 1,
        4, 7, 3, 3, 6, 8, 0, 2, 5, 7, 8, 5, 7, 1, 8, 5, 7, 0, 1, 1, 0, 3, 4, 2,
        0, 0, 5, 1, 2, 5, 8, 9, 6, 2, 1, 3, 9, 3, 7, 3, 8, 0, 2, 7, 7, 8, 9, 1,
        0, 7, 3, 4, 1, 8, 7, 5, 5, 3, 0, 4, 9, 3, 2, 2, 0, 3, 7, 8, 8, 2, 5, 3,
        4, 9, 5, 5, 5, 7, 5, 5, 8, 0, 2, 2, 3, 7, 7, 0, 2, 7, 9, 8, 6, 6, 8, 6,
        2, 6, 9, 5, 8, 7, 5, 7, 7, 1, 4, 3, 2, 5, 8, 4, 1, 2, 4, 3, 5, 3, 9, 7,
        1, 2, 6, 0, 1, 0, 5, 8, 4, 0, 8, 1, 4, 0, 0, 3, 6, 0, 6, 6, 1, 3, 3, 7,
        8, 5, 7, 3, 0, 3, 4, 2, 5, 1, 0, 1, 2, 0, 2, 2, 2, 5, 8, 9, 0, 9, 5, 7,
        3, 1, 2, 5, 7, 4, 1, 1, 7, 9, 5, 0, 7, 2, 7, 2, 7, 6, 8, 1, 3, 7, 3, 8,
        2, 1, 6, 5, 5, 4, 3, 3, 0, 8, 7, 1, 2, 8, 5, 4, 9, 4, 3, 2, 6, 5, 5, 9,
        9, 4, 6, 7, 0, 1, 9, 2, 3, 3, 7, 9, 4, 8, 5, 8, 8, 2, 2, 1, 5, 9, 9, 7,


In [5]:
def train(dataloader,model,loss_fn,optimizer):
    model.train() #告诉模型，我要开始训练，模型中w进行随机化操作，已经更新w，在训练过程中，w会被修改的
# pytorch提供2种方式来切换训练和测试的模式，分别是：model.train() 和 mdoel.eval()
# 一般用法是：在训练开始之前写上model.train(),在测试时写上model.eval()
    batch_size_num = 1
    for X,y in dataloader:              #其中batch为每一个数据的编号
        X,y = X.to(device),y.to(device) #把训练数据集和标签传入cpu或GPU
        pred = model.forward(X)         # .forward可以被省略，父类种已经对此功能进行了设置
        loss = loss_fn(pred,y)          # 通过交叉熵损失函数计算损失值loss
        # Backpropagation 进来一个batch的数据，计算一次梯度，更新一次网络
        optimizer.zero_grad()           # 梯度值清零
        loss.backward()                 # 反向传播计算得到每个参数的梯度值w
        optimizer.step()                # 根据梯度更新网络w参数

        loss_value = loss.item()        # 从tensor数据种提取数据出来，tensor获取损失值
        if batch_size_num %10 ==0:  # loss前的[number:100]
            print(f"loss: {loss_value:>7f} [number:{batch_size_num}]")
        batch_size_num += 1

def Test(dataloader,model,loss_fn):
    size = len(dataloader.dataset)  #10000
    num_batches = len(dataloader)  # 打包的数量
    model.eval()        #测试，w就不能再更新
    test_loss,correct =0,0
    with torch.no_grad():       #一个上下文管理器，关闭梯度计算。当你确认不会调用Tensor.backward()的时候
        for X,y in dataloader:
            X,y = X.to(device),y.to(device)
            pred = model.forward(X)
            test_loss += loss_fn(pred,y).item() #test_loss是会自动累加每一个批次的损失值
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            a = (pred.argmax(1) == y) #dim=1表示每一行中的最大值对应的索引号，dim=0表示每一列中的最大值对应的索引号
            b = (pred.argmax(1) == y).type(torch.float)
    test_loss /= num_batches #能来衡量模型测试的好坏
    correct /= size  #平均的正确率
    print(f"Test result: \n Accuracy:{(100*correct)}%, Avg loss:{test_loss}")

In [ ]:

epochs = 5
for t in range(epochs):
    print(f"epoch {t+1}\n---------------")
    train(dataloader,model,loss_fn,optimizer)#训练1次完整的数据。多轮训练
    Test(test_dataloader,model,loss_fn)
print("Done!")


epoch 1
---------------
loss: 0.841049 [number:10]
loss: 0.391169 [number:20]
